### P1-Preprocesamiento

    *Tokenizacion 
    *Eliminacion de signos de puntuacion
    *Eliminacion de Stopwords
    *Lematizacion
    *Stemming
    *Normalizar (minusculas)
    *Bag of Words
    *TF-IDF
    

#### Importamos las librerias a utilizar 

In [29]:
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

import spacy
nlp = spacy.load('es_core_news_sm')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ianca\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ianca\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
texto = pd.read_csv("./datasets/sentiment_analysis_dataset.csv")
texto = texto["text"]

#### Creamos la funcion de tokenizar

In [13]:
def tokenize_text(text):
    tokens = []
    current_word = ''

    for char in text:
        if char.isalnum() or char == "'":
            current_word += char
        elif current_word:
            tokens.append(current_word.lower())
            current_word = ''
    
    if current_word:
        tokens.append(current_word.lower())

    return tokens

allTokens = []
for row in texto:
    tokens = tokenize_text(row)
    allTokens.append(tokens)
    
# print(allTokens) #Funciona


#### Definimos nuestras Stopwords en un arreglo

In [14]:
stopwords_espanol = [
    "de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las", "por", "un", "para", "con", "no",
    "una", "su", "al", "lo", "como", "más", "pero", "sus", "le", "ya", "o", "este", "sí", "porque", "esta",
    "entre", "cuando", "muy", "sin", "sobre", "también", "me", "hasta", "hay", "donde", "quien", "desde",
    "todo", "nos", "durante", "todos", "uno", "les", "ni", "contra", "otros", "ese", "eso", "ante", "ellos",
    "e", "esto", "mí", "antes", "algunos", "qué", "unos", "yo", "otro", "otras", "otra", "él", "tanto",
    "esa", "estos", "mucho", "quienes", "nada", "muchos", "cual", "poco", "ella", "estar", "estas", "algunas",
    "algo", "nosotros", "mi", "mis", "tú", "te", "ti", "tu", "tus", "ellas", "nosotras", "vosotros", "vosotras",
    "os", "mío", "mía", "míos", "mías", "tuyo", "tuya", "tuyos", "tuyas", "suyo", "suya", "suyos", "suyas",
    "nuestro", "nuestra", "nuestros", "nuestras", "vuestro", "vuestra", "vuestros", "vuestras", "esos", "esas",
    "estoy", "estás", "está", "estamos", "estáis", "están", "esté", "estés", "estemos", "estéis", "estén", "estaré",
    "estarás", "estará", "estaremos", "estaréis", "estarán", "estaría", "estarías", "estaríamos", "estaríais",
    "estarían", "estaba", "estabas", "estábamos", "estabais", "estaban", "estuve", "estuviste", "estuvo", "estuvimos",
    "estuvisteis", "estuvieron", "estuviera", "estuvieras", "estuviéramos", "estuvierais", "estuvieran", "estuviese",
    "estuvieses", "estuviésemos", "estuvieseis", "estuviesen", "estando", "estado", "estada", "estados", "estadas",
    "estad", "he", "has", "ha", "hemos", "habéis", "han", "haya", "hayas", "hayamos", "hayáis", "hayan", "habré",
    "habrás", "habrá", "habremos", "habréis", "habrán", "habría", "habrías", "habríamos", "habríais", "habrían",
    "había", "habías", "habíamos", "habíais", "habían", "hube", "hubiste", "hubo", "hubimos", "hubisteis", "hubieron",
    "hubiera", "hubieras", "hubiéramos", "hubierais", "hubieran", "hubiese", "hubieses", "hubiésemos", "hubieseis",
    "hubiesen", "habiendo", "habido", "habida", "habidos", "habidas", "soy", "eres", "es", "somos", "sois", "son",
    "sea", "seas", "seamos", "seáis", "sean", "seré", "serás", "será", "seremos", "seréis", "serán", "sería", "serías",
    "seríamos", "seríais", "serían", "era", "eras", "éramos", "erais", "eran", "fui", "fuiste", "fue", "fuimos",
    "fuisteis", "fueron", "fuera", "fueras", "fuéramos", "fuerais", "fueran", "fuese", "fueses", "fuésemos",
    "fueseis", "fuesen", "sintiendo", "sentido", "sentida", "sentidos", "sentidas", "siente", "sentid", "tengo",
    "tienes", "tiene", "tenemos", "tenéis", "tienen", "tenga", "tengas", "tengamos", "tengáis", "tengan", "tendré",
    "tendrás", "tendrá", "tendremos", "tendréis", "tendrán", "tendría", "tendrías", "tendríamos", "tendríais",
    "tendrían", "tenía", "tenías", "teníamos", "teníais", "tenían", "tuve", "tuviste", "tuvo", "tuvimos", "tuvisteis",
    "tuvieron", "tuviera", "tuvieras", "tuviéramos", "tuvierais", "tuvieran", "tuviese", "tuvieses", "tuviésemos",
    "tuvieseis", "tuviesen", "teniendo", "tenido", "tenida", "tenidos", "tenidas", "tened"
]



#### Creamos la funcion de filtrado de stopwords

In [17]:
def filtrar_stopwords(tokens, stopwords):
    tokens_filtrados = [token for token in tokens if token.lower() not in stopwords]
    return tokens_filtrados

allTokensFiltrados = []

for token in allTokens:
    tokensFiltrados = filtrar_stopwords(token, stopwords_espanol)
    allTokensFiltrados.append(tokensFiltrados)

#print(allTokensFiltrados) #Funciona

#### Creamos la funcion para elimiar acentos 

In [22]:
def quitar_acentos(tokens):
    tokensSinAcentos = [unidecode(token) for token in tokens]
    return tokensSinAcentos


allTokensSinAcentos = []
for token in allTokensFiltrados:
    tokensSinAcentos = quitar_acentos(token)
    allTokensSinAcentos.append(tokensSinAcentos)
    
#print(allTokensSinAcentos) #Funciona

#### Creamos a funcion que me lematiza los tokens

In [24]:
def lematizar(tokens):
    lemas = []
    for token in tokens:
        doc = nlp(token)
        for token in doc:
            lemas.append(token.lemma_)
    return lemas

allTokensLematizados = []
for token in allTokensSinAcentos:
    lemas = lematizar(token)
    allTokensLematizados.append(lemas)

#print(allTokensLematizados) #Funciona

#### Creamos la funcion que nos hace el Stemming

In [25]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')

def stemming(tokens):
    stems = [stemmer.stem(token) for token in tokens]
    return stems

allTokensStemming = []
for token in allTokensSinAcentos:
    raices = stemming(token)
    allTokensStemming.append(raices)
    
#print(allTokensStemming) #Funciona

#### Creamos la funcion de las bag of words para vectorizar

In [27]:
def bag_of_words(documents):
    # Creamos un conjunto para almacenar todas las palabras únicas en los documentos
    vocabulary = set()
    for document in documents:
        vocabulary.update(document)

    # Creamos un diccionario para almacenar las frecuencias de las palabras en cada documento
    bow = []
    for document in documents:
        doc_bow = {}
        for word in vocabulary:
            doc_bow[word] = document.count(word)
        bow.append(doc_bow)

    return bow, list(vocabulary)

# Ejemplo de uso:

bow_representation, vocabulary = bag_of_words(allTokensSinAcentos)
# print("Bag of Words representation:")
# for doc_bow in bow_representation:
#     print(doc_bow)
# print("\nVocabulary:", vocabulary)


#### Funcion TF-IDF

In [41]:
def calcular_tfidf(documentos_tokens):
    # Convertir la lista de listas de tokens en documentos de texto nuevamente
    documentos_texto = [" ".join(tokens) for tokens in documentos_tokens]
    
    # Inicializar el vectorizador TF-IDF
    vectorizador_tfidf = TfidfVectorizer()

    # Calcular TF-IDF
    matriz_tfidf = vectorizador_tfidf.fit_transform(documentos_texto)

    # Obtener los nombres de las características (palabras)
    palabras = vectorizador_tfidf.get_feature_names_out()

    # Crear un DataFrame para visualizar los resultados
    df_tfidf = pd.DataFrame(matriz_tfidf.toarray(), columns=palabras)

    return df_tfidf, documentos_texto

documentos_tokens = [
    ["este", "es", "el", "primer", "documento"],
    ["este", "documento", "es", "el", "segundo", "documento"],
    ["y", "este", "es", "el", "tercer", "documento"],
    ["es", "este", "el", "primer", "documento"]
]

# Calcular TF-IDF
resultado_tfidf, doc = calcular_tfidf(allTokensSinAcentos)
resultado_prueba_tfidf, d = calcular_tfidf(documentos_tokens)

# Imprimir el resultado
print(resultado_prueba_tfidf)


   documento        el        es      este    primer   segundo    tercer
0   0.398961  0.398961  0.398961  0.398961  0.602761  0.000000  0.000000
1   0.612215  0.306107  0.306107  0.306107  0.000000  0.586591  0.000000
2   0.361028  0.361028  0.361028  0.361028  0.000000  0.000000  0.691835
3   0.398961  0.398961  0.398961  0.398961  0.602761  0.000000  0.000000


In [46]:
archivo_csv_original = './datasets/sentiment_analysis_dataset.csv'
archivo_csv_salida = './datasets/sentiment_analysis_dataset_modificado.csv'

df_original = pd.read_csv(archivo_csv_original)

df_original['text'] = doc
df_original.to_csv(archivo_csv_salida, index=False)

print(df_original['text'])

0                       termine bien abrumado despues hoy
1                                         siento abrumado
2       siento abrumado cantidad cosas quiero dibujar ...
3       salvador unica persona abrumado versiones nadi...
4                         denme helado ando full abrumado
                              ...                        
2585    podemos vivir miedo manejen borrachos dejen us...
2586                           vida constante miedo exito
2587    esquizofrenia mente dividida miedo realidades ...
2588       miedo da ver como desapareces mundo temo mundo
2589       saltando apoco ala pile agua xd miedo relajado
Name: text, Length: 2590, dtype: object
